In [1]:
import os
import pdfkit
import pandas as pd
import hashlib
import numpy as np
import logging
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# LIST OF UNIQUE CATEGORIES THAT WAS NEEDED FOR THE DATASET.
titles = pd.read_csv(r'C:\Users\srika\OneDrive\Documents\York\Sem-2 york\MBAN 6090 - Analytics Consulting Project\webscrape\communitech\jobs.csv')
job_list =titles['Title'].unique()
job_list = job_list.tolist()
job_list

['Commercial Account Executive',
 'Director of Sales, Enterprise Acquisition',
 'Senior Software Developer | Développeur logiciel sénior',
 'Chemistry Co-op',
 'Senior Talent Acquisition Partner (15 months contract)',
 '(Senior) Research Scientist - Statistical Genetics',
 'Technical Analyst',
 'Accounting Associate - 6 months',
 'Client Support Specialist',
 'Personal Lines Sales Broker - BC',
 'Personal Lines Sales Broker - Alberta',
 'Senior Software Engineer - Front-end specialist',
 'Sales Development Representative - Commercial',
 'Senior Full Stack Engineer - Audiences, Inventory & Pixels',
 'Marketing Web Designer',
 'Project Assistant (6-month term/remote)',
 'Bilingual Junior Account Manager (Canada, Remote)',
 'Solutions Engineer',
 'Senior Manager, HR & Organization Transformation Consultant',
 'Technical Project Manager',
 'Senior Product Designer (Contract - 24 months)',
 'Senior Data Scientist',
 'Community Marketing Specialist, EMEA',
 'Product Support Technician',
 'Re

In [10]:
# Splitting the titles into required format
job_list = [title.split(' - ')[0] for title in job_list]
job_list = [title.split(' (')[0] for title in job_list]
job_list = [title.split(' | ')[0] for title in job_list]
job_list = [title.split(', ')[0] for title in job_list]
job_list = [title.split(' / ')[0] for title in job_list]
job_list = [title.lower() for title in job_list]
job_list = [title.replace('(canada) ', '').replace('(us) ', '').replace('(senior) ', '').replace('(マーケティングマネージャー)','').strip() for title in job_list]
job_list = list(set(job_list))  
job_list.sort()
job_list

['- software engineer position',
 '.net developer',
 '.net full stack developer',
 '.net software developer',
 '2023 aquamobile swim instructor and/or lifeguard',
 '3d designer',
 '3d printing test engineer',
 '442a electrician',
 '533650-field service',
 '539242-enterprise customer care',
 'aba clinic software implementation specialist',
 'abm marketing manager',
 'accessibility tester',
 'account based marketing specialist',
 'account coordinator',
 'account development executive',
 'account development representative',
 'account director',
 'account executive',
 'account executive -montreal',
 'account executive-enterprise sales',
 'account executives',
 'account lead',
 'account manager',
 'account manager technique',
 'account payable lead',
 'account relationship manager',
 'account support officer',
 'account support officer-abuja',
 'accountant',
 'accounting',
 'accounting associate',
 'accounting clerk',
 'accounting manager',
 'accounting operations intern',
 'accounting sol

In [11]:
# Number of jobs finalised
len(job_list)

2845

In [ ]:
# Set the URL and headers
payload = {}
headers = {
  'authority': 'www.livecareer.com',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'accept-language': 'en-US,en;q=0.9',
  'cache-control': 'no-cache',
  'cookie': 'x-georegion=295,CA,ON,TORONTO,,,,,,43.67,-79.42,EST,M3H+M3M+M4B+M4C+M4E+M4G+M4H+M4J+M4K+M4L+M4M+M4N+M4P+M4R+M4S+M4T+M4V+M4W+M4X+M4Y+M5A+M5B+M5C+M5E+M5G+M5H+M5J+M5K+M5L+M5M+M5N+M5P+M5R+M5S+M5T+M5V+M5W+M5X+M6A+M6B+M6C+M6E+M6G+M6H+M6J+M6K+M6L+M6M+M6N+M6P+M6R+M6S+M7A+M7Y+M9M+M9N+M9P+M9W,NA,,,802,vhigh,5000; vssessionuid=edc7e3c7-125e-4517-bc17-1e87a377baa7; vstr=44e7a059-9df4-40eb-a9bc-a214f1ba0e51; vsuid=83923cf0-d086-4342-ae22-ad7c2ef0b072; ref=14; visitinfo=[City,North York]&[State,ON]&[Country,CA]&[PostalCode,M3J]&[BrowserName,Chrome]&[BrowserVersion,120]&[DeviceType,]&[OSName,Windows]&[DeviceModel,Unknown]&[OSVersion,10.0]; vsutms=8846de03-0f2b-4b70-870c-5771d4b61295#44e7a059-9df4-40eb-a9bc-a214f1ba0e51#83923cf0-d086-4342-ae22-ad7c2ef0b072#1703736192#https://google.com#||||; vsintentcd=RSGNRL; ci_id=dc617d47-a16f-442b-afa5-t18cae972f2d-e76f342896dc; ARRAffinity=04e4ae4662bb6d686af058e5e5dc2f336bdaef569f9a9b7df71508476ed2041f; ARRAffinitySameSite=04e4ae4662bb6d686af058e5e5dc2f336bdaef569f9a9b7df71508476ed2041f; Auth=tZD-nTtxJSfHBh-UPP6UEX5aGGD7RT7ePjtoI616JXRZKcfA6NYzXpySLcHL86E1f7-xi8oqZI3iiiUetJkCbHhKP4qD3pfsAVmyRZGtaEEnX2OAGWtENeOeef4Vi3yXfH8IzRCgqUQL1NUnf-nbmifEQN2wA087ivT37M-nJ5WPhFIOq593fOhVJ-uCbmPMKTvKVhHUpewTgzv61TyPMOtnxNMnIpLeYrYogpSWIiXcGWcWp_n8U3t6XZ-GMqF1SWc-VwK2H6rfJuBUwRnM1izratGTxaP7qFTIuc4RzF2ZhjsAs2vxOLxorEEwv1RnvAVEgE0j3rP4K35YOYYE0aKakyNPHMnE7YON4tgZC3cRS6kHID6L2Ag__aiYGDXx4sX5mFzDZelkQDZ0VAppdK1sqgh2GtoPX0HhYEB0UD3mgzcuktG6NkZIwZ6k-ucIz2nRniqU_UqrPRuXl4YjcLxNtBSR6UitAhbzWK8cMBQpuHsUSAQaaptw8SDorKnbQd-UFdC1tA04RdfzApok_q7urdLwICWUSWNk6fvy09HFuJJorlKipsCJWCLBrMXVud6BwgxcmRzX0DB6N9TMlCHmM0i1puNmzGD_FqPq0Sc6nffmPWTw9_ei7vG7HEjQF8jPMdQa5xKOkHWmEfwDqmg5P84iBIpRgFm18Ns5QWhEw5hIBj0w4vtEQjrfCz5HDMVeHxbnvXuHsJF2dlsN_IvfRlQI85P0jwpjEbyoXZICmAvVeBHrT7p1WrtxQFoOu8xesRgnXZ3RCvvGsteq0D0j_ltMJhSB8qrQVgE2ZTg1G7ed_iyZDku1UdoSbU-qJYut7ZA7IXZ0MgvI30aBFAbWPQ61bAzx993ax6s65vHyklRP06IUEN9rXPaP9S6wlA9lcSG18tZfHYRv9_k3I_08DBmfwsUzkwK4igN_TTZxk7cNvsewUNwVWMbCUI8WfU8Ni7Y911XiwmGzj1Y9qya6rYcD_leyI-RRiTmh1-4v_o9URlT1z66Y0ShhippQ; UserStatus={"IsUserLoggedIn":false,"User":{"Role":0,"UserId":"847d4a5f-9a42-48c9-8eac-450c86379d15","AccDisplayName":"Guest","CreatedOn":"12/28/2023 5:30:28 AM"}}; acc_session={ec77e065-8bed-4bde-9399-4997c66294a9}; isFirstTimeUser=0; DocumentID=fa1d4d7e-1452-90c3-e316-ed51b5b09287; bm_sz=A5496B0C2D3CBDE33F6425F7AEC79CCD~YAAQ5mUzuC5KM2eMAQAAhcMIsRaPm6BXjg5oIsXlVJeqwXdsbDy4bGV0eCuk2taG7rc9fSM1fzzXFSRiE297o4KA4H/ym/ALT6ZgkqN5X7z+gruIkmV4eJ41uZaIGT1Wh9K/PO0iVD3omBh7Ni2Lu1RAY6UMVWLBnav7gT8nDxQpK6jbDUdx+9gy1SMNb1sIiIxIituiC/FQAB191zlwz1XnxaSkb5sNz1Lsgb3wl/Euu2Qa+RRcNpAH4E48Mv/A8m0Cl+MphGLK/rflwH3yFOjT7ZvcCiCGZxoad4eFzlWZTQq+WU9w~3420997~3551554; _abck=3220C70AB729592693CB0CF1DBD0D013~0~YAAQ5mUzuAFPM2eMAQAAvToJsQveF1VnQN0qe0HXz8EwMG7ECaadmMc+Z+DMvmB4P8iMehswJXpR0YHqn4NJfOq18zh/xWtNH4N1HS4WsOQuI8DUqdJhPqx9oN6L+h9vYT8p6BfiEecn680QH/AYuE+WjjV3qtjhq4ZFdWROo4wQNFYLGh2uRTvVNiOtK87N/kfQYw+OU1GiAFZog6q80AczDedYEsSMChkj8ju6bjDq8ex8s2Xb6GgW0BHnx2jxONMp1JY4wnT4FxeByE3ZJxM4WUMPMKO+dgFnNOxQkwFu9nC5NCUvrArLH9thY6Hsg+ssmJsEaOQE+wnl6nUQVUTahaY9AYF/IQhRW1lh3PAmMxNYwQRYZZ280uZGRTzv4JLNhCuUMG4ZQMoPCpwyre6v8ROBa9K055S6qg==~-1~-1~-1; bm_mi=7E3B1A39D0786EDDC1250549CCD493C4~YAAQ5mUzuLZTM2eMAQAAZqIJsRYeOVBlszn/AmZi7p5SavD3N3qOPMqu8u1JGviVIdF77HX8uyWHzcVMkdp48XnZrdwVOdpgcRbvZO2vxQYJftHYchvHfQFSR/qp2QIJEZzm18Us9W7BO8yIlYTpLH8xDLw20KfTN66P4bhKupRcOCdmDghqk69TQyN5LeRFTuUGt/a9S7YxzCZWMDko3A/4ywXgcfOJA++31EpFhxW17LK9frRQmroorMvWSvrj1+qlCpbfOYU3cAbRBohKY16cVvb2Vn/J2aPGchrMHyf0WtqElW6jgHVbwBsYGJ5Do/yC4sUBYfqanvtm~1; bm_sv=6F5884ED906C4E66C9D8DEAC55CF9ECB~YAAQ5mUzuEtUM2eMAQAANLQJsRb9JOVI7QlRAd1iKn2JbP2Gxr8LV/4RwA9jvpGD6AAizF7Hv2//R94mQbZr2AO1InFXkV2KgyBc6kKvGhrykUPcYWknumUeig1vf7t/0+Qv94pO1Kq5e4zWLC/H+bZv7B6ovpjvEWQvoLzAJhB2hwdTg5TnEXcuP24nXEbYFbMWrphrWkNV+4GUwWXgUtPgIBPof912KTKIt1/ku8f36BD0xCvnZtSETa9qWO7Ds1hzEUU=~1; ak_bmsc=F75787051AB05D2549E2D10A9E447D1F~000000000000000000000000000000~YAAQ5mUzuIlXM2eMAQAAbBAKsRYG0Jc6ZAVZkigfZkpC4RHX4NXg90EryYruj7aA4DUh227H6WEpsqXSzBY+v6xapJgz9wB2wMWAaj+j8GBiT+IRqm4Rqe7LWO918BRmCGGLcEd4G0DyxLqSaGRX0A7tgzkU6Y1uwpSr9obTaFIoQYKTCmt9rBv/0FKHMhH39ZmK6jkAZf5vzPhMAKrDiywdNxKGcvlZ04ODRsz8nJJfWbPWAMlfSxxDUu8cvw2yX+17wYZCqQ0RSFmxmar9jCmxvwCeSF4njAYdh40fSuKh6InfBE6Tldtnzj9rsELI7RMZzK8TYMha/5k9G6az1ikru5ZWXi3AObMYQglYATPEIAq6SCul+ce5UcZKuSJGOdBItH4B0peeiCWHN/4DHeR8uFglXEWPy1uHs2hHPTWpQYlLA6vOH2CFW4c=; TtfbData={"countryCD":"CA","expId":"","isCache":true,"reqTst":1703777881000,"resTst":1703777881000}; _abck=3220C70AB729592693CB0CF1DBD0D013~-1~YAAQ5mUzuHK0M2eMAQAAAoIUsQtGXtxj9d2AV6PC5f8yupKPSFIAjPklqVyd4CA4yaV5/bEszfXXMi/600kY5pB90LyhVIRAYbE8sRAeUYPFShvgcmkNUktT995LhRUSnoqg7FINqEd1MEUdSqYJwan+gblIUIALw4jNLD6MNQWxi69e3/LGURA+ZXmJbA4mIvi2QbyUs1/JP/z+8Um1YbaDSmNl3geKxZP/lRxJQ86ng17jK1qhvgjkookpR/bN7VKjqwXbufTvg85JB2wPL0LxQJH11jIym769AdSwLDpqmj75kcFPR0azEyNGE8p0Y3M5QJBjCApGg9M1Yko9VbyBB43gmo3OryASJErfDqT9aQxoZAi7zq+e1CHdmupBVLEeiRFwxnlQ2tJHPlriiRuevTnBTlVqRC63Yw==~0~-1~-1',
  'dnt': '1',
  'pragma': 'no-cache',
  'referer': 'https://www.livecareer.com/resume-search',
  'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

In [ ]:
# set up logging
logging.basicConfig(level=logging.INFO)

In [ ]:
# creating an empty list to store the data
total_data = []

In [12]:
for job in job_list:
    JOB = job.lower()
    link_counter = 0  
    PAGE = '1' 

    URL = "https://www.livecareer.com/resume-search/search?jt=" + JOB + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
    response = requests.get(URL, headers=headers, data=payload)

    if response.status_code != 200:
        logging.warning(f"Failed to fetch data for {URL}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    resume_items = soup.find_all('a', class_='sc-1dzblrg-0 caJIKu sc-1os65za-2 jhoVRR')

    for item in resume_items:
        if link_counter >= 4:
            break 

        title = item.find('p', class_='title').get_text(strip=True)
        company = item.find('p', class_='company-title').get_text(strip=True)
        location = item.find('p', class_='company location').get_text(strip=True)
        updated = item.find('p', class_='last-updated').get_text(strip=True)
        resume_url = item['href']

        total_data.append({
            'Title': title,
            'Company': company,
            'Location': location,
            'Last Updated': updated,
            'URL': f"https://www.livecareer.com{resume_url}"
        })

        link_counter += 1 
        
# convert to Dataframe
df = pd.DataFrame(total_data)
df.head()

,Title,Company,Location,Last Updated,URL
0,software engineer of iot cloud,Software Engineer of IoT Cloud@ Salesforce,"Salesforce | Dublin, California","Last Updated OnOctober 28, 2016",https://www.livecareer.com/resume-search/r/sof...
1,senior software engineer,Senior Software Engineer@ Cognizant Technology...,"Cognizant Technology Solutions | Redmond, Wash...","Last Updated OnMarch 27, 2018",https://www.livecareer.com/resume-search/r/sen...
2,software engineer,Software Engineer@ ROLTA India LTD,"ROLTA India LTD | New York, New York","Last Updated OnDecember 13, 2016",https://www.livecareer.com/resume-search/r/sof...
3,senior software engineer,Senior Software Engineer@ Verizon,"Verizon | Coppell, Texas","Last Updated OnMay 13, 2015",https://www.livecareer.com/resume-search/r/sen...
4,.net developer,.Net developer@ Wells Fargo,"Wells Fargo | Ozone Park, New York","Last Updated OnMarch 22, 2016",https://www.livecareer.com/resume-search/r/-ne...


In [13]:
# total shape of the dataframe
df.shape

(11179, 5)

In [ ]:
# drop duplicates
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.shape

In [ ]:
# Add empty columns for Raw_html and Resume
df['Raw_html'] = ""
df['Resume'] = ""

In [ ]:
# Set the path to your Chrome webdriver executable 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("webdriver.chrome.driver=path/to/chromedriver.exe")

In [ ]:
# Initialize the Chrome webdriver with the specified options
driver = webdriver.Chrome(options=chrome_options)

In [15]:
for i in range(df.shape[0]):
    url = df['URL'].iloc[i]  
    driver.get(url)
    
    try:
        # Wait for the specific element to be loaded. In this case, the element with ID "server-resume-template"
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#server-resume-template > div > div"))
        )
        
        # Get the page source and process it
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find the relevant div containing the resume
        resume_div = soup.select_one("#server-resume-template > div > div")
        raw_html = str(resume_div) if resume_div else ""
        resume_text = resume_div.get_text(separator=" ", strip=True) if resume_div else ""

        # Update the DataFrame
        df.at[i, 'Raw_html'] = raw_html
        df.at[i, 'Resume'] = resume_text

    except TimeoutException:
        print(f"Timeout while loading {url}")
    except Exception as e:
        print(f"An error occurred while processing URL {url}: {e}")
        df.at[i, 'Raw_html'] = ""
        df.at[i, 'Resume'] = ""


Timeout while loading https://www.livecareer.com/resume-search/r/network-security-administrator-firewall-engineer-41df126cfcca4d76a02729ad9b3a9e8b
Timeout while loading https://www.livecareer.com/resume-search/r/senior-customer-success-manager-1c3e10006b9f4cafb7cfd8fc955dc4de


In [16]:
df = df[['Title','Raw_html', 'Resume']]
df.head()

,Title,Raw_html,Resume
0,software engineer of iot cloud,"<div class=""document fontsize fontface vmargin...","Jessica Claire Montgomery Street , San Francis..."
1,senior software engineer,"<div class=""document fontsize fontface vmargin...","Jessica Claire Montgomery Street , San Francis..."
2,software engineer,"<div class=""document fontsize fontface vmargin...","Jessica Claire Montgomery Street , San Francis..."
3,senior software engineer,"<div class=""document fontsize fontface vmargin...","Jessica Claire Montgomery Street , San Francis..."
4,.net developer,"<div class=""document fontsize fontface vmargin...","Jessica Claire Montgomery Street , San Francis..."


In [17]:
# df.to_csv('resume_links_231230_2.csv', index=False)
df.to_csv('resume_data_231230.csv', index=False)

In [ ]:
# Sample the DataFrame
sample = df.sample(1)
sample.reset_index(inplace=True, drop=True)

In [ ]:
# Directory for saving the PDF
parent_dir = r'C:\Users\srika\OneDrive\Documents\York\Sem-2 york\MBAN 6090 - Analytics Consulting Project\webscrape\communitech'  # Replace with your desired path
label = 'sample_pdf'  # Folder name
path = os.path.join(parent_dir, label)
os.makedirs(path, exist_ok=True)

In [ ]:
# Specify the path to wkhtmltopdf executable
config = pdfkit.configuration(wkhtmltopdf=r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe")

In [18]:
# Generate PDF
pdf_path = os.path.join(path, sample['Title'][0].replace('/', '_') + ".pdf")  # Replacing '/' to prevent directory traversal
pdfkit.from_string(sample['Raw_html'][0], pdf_path, configuration=config)

print(f"PDF saved at {pdf_path}")

PDF saved at C:\Users\srika\OneDrive\Documents\York\Sem-2 york\MBAN 6090 - Analytics Consulting Project\webscrape\communitech\sample_pdf\chef de partie.pdf
